In [ ]:
import pandas as pd
from src.all_in_one import *
threshold = "2022-12-31"
upper_threshold = "2024-01-01"

In [ ]:
scr_dataset = Dataset(config_file, scr_sheet)
scr = pd.read_csv(scr_path)
unique_scr = scr.drop_duplicates(subset='rid')

In [ ]:
int_dataset = Dataset(config_file, int_sheet)
intake = pd.read_csv(int_path)
dtype_trans(intake, int_dataset)
# intake.insert(4, "service", None)
intake.sort_values(by=['nt_s1', 'nt_s2', 'nt_s3', 'nt_re'], ascending=True)
unique_intake = intake.drop_duplicates(subset='rid', keep='last')

In [ ]:
# intake_23 = filter_df_by_date(intake, ["nt_s2"], threshold=threshold)
# intake_23 = intake_23[["rid", "fcid", "service","nt_s2", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# intake_23["service"] = "int"
# intake_23.rename(columns={"nt_s2": "date"}, inplace=True)
# intake_23

In [ ]:
# intake3_23 = filter_df_by_date(intake, ["nt_s3"], threshold=threshold)
# intake3_23 = intake3_23[["rid", "fcid", "service","nt_s3", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# intake3_23["service"] = "int"
# intake3_23.rename(columns={"nt_s3": "date"}, inplace=True)
# intake3_23

In [ ]:
# reintake_23 = filter_df_by_date(intake, ["nt_re"], threshold=threshold)
# reintake_23 = reintake_23[["rid", "fcid", "service","nt_re", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# reintake_23["service"] = "re"
# reintake_23.rename(columns={"nt_re": "date"}, inplace=True)
# reintake_23

In [ ]:
# int23 = pd.concat([intake_23, intake3_23,reintake_23])
# int23

In [ ]:
# int23 = int23.sort_values(by='date')
# int23 = int23.drop_duplicates(subset="rid", keep="last")
# int23

In [ ]:
# int23 = pd.merge(int23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
# gc_dataset = Dataset(config_file, gc_sheet)
# gc = pd.read_csv(gc_path)
# dtype_trans(gc, gc_dataset)

In [ ]:
# gc23 = filter_df_by_date(gc, ['gc_s1', 'gc_s2', 'gc_s3', 'gc_s4', 'gc_s5', 'gc_s6', 'gc_s7', 'gc_s8', 'gc_s9', 'gc_10'], threshold)
# gc23 = gc23[gc23["gc_total"] >= 7]
# gc23_latestdate = gc23[['gc_s1', 'gc_s2', 'gc_s3', 'gc_s4', 'gc_s5', 'gc_s6', 'gc_s7', 'gc_s8', 'gc_s9', 'gc_10']].max(axis=1)
# gc23.insert(4, "service", "gc")
# gc23.insert(5, "date", gc23_latestdate)
# gc23

In [ ]:
# gc23 = gc23[['rid', 'fcid', 'service', 'date', 'gc_total']]
# gc23

In [ ]:
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gc23")
# gc23 = pd.read_csv(gc23_path)
# gc23

In [ ]:
# gc23 = pd.merge(gc23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')

In [ ]:
# gc23 = pd.merge(gc23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# gc23

In [ ]:
# ic_dataset = Dataset(config_file, ic_sheet)
# ic = pd.read_csv(ic_path)
# dtype_trans(ic, ic_dataset)
# ic

In [ ]:
# ic23 = filter_df_by_date(ic, ic_dataset.dvars, threshold)
# ic23_latestdate = ic23[ic_dataset.dvars].max(axis=1)
# ic23.insert(4, "service", "ic")
# ic23.insert(5, "date", ic23_latestdate)
# ic23 = ic23[['rid', 'fcid', 'service', 'date', 'ic_total']]
# ic23 = pd.merge(ic23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ic23 = pd.merge(ic23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# ic23

In [ ]:
# int23_path = os.path.join(root, data, processed, "unvfvt23", "intBeneficiaries23.csv")
# int23.to_csv(path, index=False)
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gcBeneficiaries23.csv")
# gc23.to_csv(gc23_path, index=False)

In [ ]:
# pd.read_csv(int23_path)

In [ ]:
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gcBeneficiaries23.csv")
# pd.read_csv(gc23_path)

In [ ]:
psfua_dataset = Dataset(config_file, ptfua_sheet)
psfua = pd.read_csv(psfua_path)
psfua = dtype_trans(psfua, psfua_dataset)

In [ ]:
psfua323 = filter_df_by_date(psfua, ['fu_3m'], threshold)
psfua323.insert(4, "service", "psfua3")
# psfua323

In [ ]:
psfua323

In [ ]:
psfua623 = filter_df_by_date(psfua, ['fu_6m'], threshold)
psfua623 = filter_df_by_date(psfua623, ['fu_6m'], upper_threshold, comparison_type='b')
psfua623.insert(4, "service", "psfua6")
psfua623

In [ ]:
psfua1223 = filter_df_by_date(psfua, ['fu_12m'], threshold)
psfua1223.insert(4, "service", "psfua12")
psfua1223

In [ ]:
psfua323 = psfua323[['rid', 'fcid', 'service', 'fu_3m']].copy()
psfua323.rename(columns={'fu_3m': 'date'}, inplace=True)
psfua623 = psfua623[['rid', 'fcid', 'service', 'fu_6m']].copy()
psfua623.rename(columns={'fu_6m': 'date'}, inplace=True)
psfua1223 = psfua1223[['rid', 'fcid', 'service', 'fu_12m']].copy()
psfua1223.rename(columns={'fu_12m': 'date'}, inplace=True)
psfua23 = pd.concat([psfua323, psfua623, psfua1223])

In [ ]:
psfua23.tocsv()